# Part 3, Multilayer Perceptron Neural Network

In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches


%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [2]:
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [3]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

# np.log(train['loss'] + 200) provides
# a better score, but let's keep it simple now
train_y = np.array(train['loss'])

print (train_x.shape)
print (train_y.shape)

(188318, 1153)
(188318,)


In [ ]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [ ]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adam','adadelta']),
         'wdecay':hp.choice('wdecay', np.linspace(0.00001,0.1,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], init='he_normal',
                        W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], init='he_normal',W_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.compile(loss='mae', optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.4210526315789474, 'hidden1_units': 455, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 272, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 69, 'optimizer': 'adadelta', 'wdecay': 0.032330000000000005}
Fold 0, MAE: 1163.148839039124
Fold 1, MAE: 1159.9899238609737
Fold 2, MAE: 1140.9771315408389
3-fold CV score: 1154.7052981469787
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.43333333333333335, 'hidden2_units': 134, 'hidden3_dropout': 0.10000000000000001, 'hidden3_units': 30, 'optimizer': 'adadelta', 'wdecay': 0.082830000000000001}
Fold 0, MAE: 1181.2090776051534
Fold 1, MAE: 1175.7242010975745
Fold 2, MAE: 1149.1617817128606
3-fold CV score: 1168.698353471863
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 113, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 22, 'optimizer': 'adam', '

Fold 0, MAE: 1160.1148852933434
Fold 1, MAE: 1153.5388360438872
Fold 2, MAE: 1146.284142179796
3-fold CV score: 1153.312621172342
Model Testing: {'hidden1_dropout': 0.44210526315789478, 'hidden1_units': 325, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 258, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 59, 'optimizer': 'adadelta', 'wdecay': 0.058590000000000003}
Fold 0, MAE: 1159.533359263806
Fold 1, MAE: 1154.5401700530679
Fold 2, MAE: 1152.34695090863
3-fold CV score: 1155.473493408501
Model Testing: {'hidden1_dropout': 0.43157894736842106, 'hidden1_units': 377, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 189, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 63, 'optimizer': 'adadelta', 'wdecay': 0.081820000000000004}
Fold 0, MAE: 1162.9307342647628
Fold 1, MAE: 1177.8290937843656
Fold 2, MAE: 1148.0584482485735
3-fold CV score: 1162.9394254325673
Model Testing: {'hidden1_dropout': 0.55789473684210522, 'hidden1_units': 455, 'hidden2_dropout

Fold 0, MAE: 1163.881212321844
Fold 1, MAE: 1164.1301011722203
Fold 2, MAE: 1150.2623383121809
3-fold CV score: 1159.4245506020818
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 532, 'hidden2_dropout': 0.40000000000000002, 'hidden2_units': 251, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adadelta', 'wdecay': 0.074749999999999997}
Fold 0, MAE: 1161.926003949864
Fold 1, MAE: 1171.028653969578
Fold 2, MAE: 1147.243582406554
3-fold CV score: 1160.0660801086653
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 300, 'hidden2_dropout': 0.33333333333333337, 'hidden2_units': 127, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adam', 'wdecay': 0.0020300000000000001}
Fold 0, MAE: 1157.4905136832626
Fold 1, MAE: 1157.7959576055189
Fold 2, MAE: 1145.165175664648
3-fold CV score: 1153.4838823178097
Model Testing: {'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.

## Step 6: The final model

It took several rounds of optimization to narrow down the parameters of the model. Here are the results.

First, the architecture. The final 4-layer model uses dropout as a regularization and a batch normalization prior to each hidden layer.

<img src="http://cdn.rawgit.com/dnkirill/allstate_capstone/master/images/mlp3.svg"></td>

And this is the model itself:

In [4]:
def cross_validate_mlp(mlp_func, nfolds=5):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=1, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [9]:
from keras import regularizers
from keras.regularizers import l2
def hyper_model():
    model = Sequential()
    model.add(Dense(437, input_dim=train_x.shape[1], init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))
    
    model.add(Dense(182, init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(73, init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.233))
    
    model.add(Dense(1, init='he_normal',W_regularizer=l2(0.002)))
    model.compile(loss='mae', optimizer='adadelta')
    return model

In [10]:
cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(437, input_dim=1153, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  """
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(182, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(73, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_normal", kernel_regularizer

Train on 120523 samples, validate on 30131 samples
Epoch 1/30
120523/120523 [==============================] - 10s 87us/step - loss: 2110.2391 - val_loss: 1197.0431
Epoch 2/30
120523/120523 [==============================] - 10s 80us/step - loss: 1324.4037 - val_loss: 1175.1480
Epoch 3/30
120523/120523 [==============================] - 10s 80us/step - loss: 1300.8877 - val_loss: 1170.7676
Epoch 4/30
120523/120523 [==============================] - 10s 80us/step - loss: 1293.9524 - val_loss: 1162.6197
Epoch 5/30
120523/120523 [==============================] - 10s 79us/step - loss: 1278.0474 - val_loss: 1159.6682
Epoch 6/30
120523/120523 [==============================] - 10s 79us/step - loss: 1279.1510 - val_loss: 1154.3661
Epoch 7/30
120523/120523 [==============================] - 9s 79us/step - loss: 1275.6025 - val_loss: 1154.0299
Epoch 8/30
120523/120523 [==============================] - 10s 80us/step - loss: 1264.7699 - val_loss: 1151.0177
Epoch 9/30
120523/120523 [============

120524/120524 [==============================] - 11s 88us/step - loss: 2309.3518 - val_loss: 1204.4172
Epoch 2/30
120524/120524 [==============================] - 10s 80us/step - loss: 1328.8722 - val_loss: 1172.1232
Epoch 3/30
120524/120524 [==============================] - 10s 80us/step - loss: 1301.3722 - val_loss: 1165.6668
Epoch 4/30
120524/120524 [==============================] - 10s 80us/step - loss: 1288.8669 - val_loss: 1155.5804
Epoch 5/30
120524/120524 [==============================] - 10s 79us/step - loss: 1287.3586 - val_loss: 1153.7166
Epoch 6/30
120524/120524 [==============================] - 9s 78us/step - loss: 1277.0266 - val_loss: 1151.5029
Epoch 7/30
120524/120524 [==============================] - 10s 79us/step - loss: 1276.8890 - val_loss: 1147.0473
Epoch 8/30
120524/120524 [==============================] - 10s 80us/step - loss: 1265.8273 - val_loss: 1148.0210
Epoch 9/30
120524/120524 [==============================] - 10s 79us/step - loss: 1260.6379 - val_lo

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.